In [2]:
import pandas as pd
import numpy as np 
import ast

In [3]:
df_mov = pd.read_csv(r'C:\Users\ignac\OneDrive\Escritorio\Proyecto Individual 1\Archivos\movies_dataset.csv', sep=',')

C:\Users\ignac\AppData\Local\Temp\ipykernel_1780\1333978002.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mov = pd.read_csv(r'C:\Users\ignac\OneDrive\Escritorio\Proyecto Individual 1\Archivos\movies_dataset.csv', sep=',')


In [4]:
df_mov.shape

(45466, 24)

In [5]:
#Funcion para extraer datos anidados
def extraer(list):
    try:
        if list != '[]':
            list_aux= ast.literal_eval(list)

            return ', '.join([dic['name'] for dic in list_aux])
        else: 
            return 'no register'
        
    except:
        return 'no register'
    

In [6]:
#Funcion para extraer datos anidados de "cast"
def extraer2(list):
    try:
        if list != []:
            list_aux= ast.literal_eval(list)

            return ', '.join([dic['name'] for dic in list_aux])
        else: 
            return 'no register'
        
    except:
        return 'no register'

In [7]:
#Extraemos datos anidados en columnas solo guardando 'name'
df_mov['genres_name'] = df_mov['genres'].apply(extraer)
df_mov['companies_names'] = df_mov['production_companies'].apply(extraer)

In [8]:
#Extraemos datos anidados de las columnas "spoken_languages", "production_countries"
df_mov['spoken_languages_clean'] = df_mov['spoken_languages'].apply(extraer2)
df_mov['production_countries_clean'] = df_mov['production_countries'].apply(extraer2)

In [9]:
#Borramos las columnas
df_mov.drop(columns=['production_companies','genres','production_countries','spoken_languages'], inplace=True)

In [10]:
# Seguimos con el desanidado. Extraemos datos relevantes de la columna 'belongs_to_collection'
df_mov['belongs_to_collection'] = df_mov['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [11]:
#df_mov['id_collection'] = df_mov['belongs_to_collection'].apply(lambda x: x.get('id') if isinstance(x, dict) else 0	)
df_mov['name_collection'] = df_mov['belongs_to_collection'].apply(lambda x: x.get('name') if isinstance(x, dict) else 0	)

In [12]:
#Borramos la columna "belongs_to_collection"
df_mov.drop(columns=['belongs_to_collection'], inplace=True)

In [13]:
# Convertimos a int o float columnas y ponemos los errores en nulo

df_mov['budget'] = pd.to_numeric(df_mov['budget'], errors='coerce')

df_mov['popularity'] = pd.to_numeric(df_mov['popularity'], errors='coerce')

df_mov['runtime'] = pd.to_numeric(df_mov['runtime'], errors='coerce')

df_mov['vote_average'] = pd.to_numeric(df_mov['vote_average'], errors='coerce')

df_mov['vote_count'] = pd.to_numeric(df_mov['vote_count'], errors='coerce')

In [14]:
# Borramos las siguientes columnas: 

df_mov.drop(columns=['video', 'imdb_id','adult', 'original_title','poster_path','homepage'], inplace=True)

In [15]:
# Eliminamos filas con valores nulos en 'release_date'
df_mov = df_mov.dropna(subset=['release_date'])

In [16]:
# Reemplazamos valores nulos en 'revenue' por 0 y pasamos a int
df_mov['revenue'] = df_mov['revenue'].fillna(0).astype(int)

In [17]:
# Creamos la columna "return" y evitamos la división por cero.
df_mov['return'] = df_mov.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] != 0 else 0, axis=1)

In [18]:
# Verificamos duplicados
df_duplicados = df_mov.duplicated()
num_duplicados = df_duplicados.sum()
print(num_duplicados)

17


In [19]:
# Seleccionamos las primeras 17 filas duplicadas

duplicados = df_duplicados[df_duplicados].index[:17]

df_mov = df_mov.drop(index=duplicados)

In [20]:
# Borramos la columna "tagline" ya que tiene muchos nulos y no considero que sea un campo relevante.

df_mov.drop(columns='tagline', inplace=True)

In [21]:
df_mov.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count,genres_name,companies_names,spoken_languages_clean,production_countries_clean,name_collection,return
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,7.7,5415.0,"Animation, Comedy, Family",Pixar Animation Studios,English,United States of America,Toy Story Collection,12.451801
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104.0,Released,Jumanji,6.9,2413.0,"Adventure, Fantasy, Family","TriStar Pictures, Teitler Film, Interscope Com...","English, Français",United States of America,0,4.043035
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0,101.0,Released,Grumpier Old Men,6.5,92.0,"Romance, Comedy","Warner Bros., Lancaster Gate",English,United States of America,Grumpy Old Men Collection,0.000000
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156,127.0,Released,Waiting to Exhale,6.1,34.0,"Comedy, Drama, Romance",Twentieth Century Fox Film Corporation,English,United States of America,0,5.090760
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911,106.0,Released,Father of the Bride Part II,5.7,173.0,Comedy,"Sandollar Productions, Touchstone Pictures",English,United States of America,Father of the Bride Collection,0.000000


In [22]:
# Contamos los valores nulos de cada columna
df_mov.isnull().sum()

budget                          3
id                              0
original_language              11
overview                      941
popularity                      3
release_date                    0
revenue                         0
runtime                       249
status                         83
title                           3
vote_average                    3
vote_count                      3
genres_name                     0
companies_names                 0
spoken_languages_clean          0
production_countries_clean      0
name_collection                 0
return                          3
dtype: int64

In [23]:
# Borramos nulos 
df_mov.dropna(inplace = True)

In [24]:
# Chequeamos nulos
df_mov.isnull().sum()

budget                        0
id                            0
original_language             0
overview                      0
popularity                    0
release_date                  0
revenue                       0
runtime                       0
status                        0
title                         0
vote_average                  0
vote_count                    0
genres_name                   0
companies_names               0
spoken_languages_clean        0
production_countries_clean    0
name_collection               0
return                        0
dtype: int64

In [25]:
# Convertimos la columna "release_date" a tipo datatime 

df_mov['release_date'] = pd.to_datetime(df_mov['release_date'])


In [26]:
# Creamos la columna "release_year"
df_mov['release_year'] = df_mov['release_date'].dt.year

In [27]:
# Visualizamos el dataframe
df_mov.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count,genres_name,companies_names,spoken_languages_clean,production_countries_clean,name_collection,return,release_year
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,7.7,5415.0,"Animation, Comedy, Family",Pixar Animation Studios,English,United States of America,Toy Story Collection,12.451801,1995
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104.0,Released,Jumanji,6.9,2413.0,"Adventure, Fantasy, Family","TriStar Pictures, Teitler Film, Interscope Com...","English, Français",United States of America,0,4.043035,1995
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0,101.0,Released,Grumpier Old Men,6.5,92.0,"Romance, Comedy","Warner Bros., Lancaster Gate",English,United States of America,Grumpy Old Men Collection,0.000000,1995
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156,127.0,Released,Waiting to Exhale,6.1,34.0,"Comedy, Drama, Romance",Twentieth Century Fox Film Corporation,English,United States of America,0,5.090760,1995
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911,106.0,Released,Father of the Bride Part II,5.7,173.0,Comedy,"Sandollar Productions, Touchstone Pictures",English,United States of America,Father of the Bride Collection,0.000000,1995


In [28]:
#Seguimos con el dataset de credits

df_creditos = pd.read_csv(r'C:\Users\ignac\OneDrive\Escritorio\Proyecto Individual 1\Archivos\credits.csv', sep=',')

In [29]:
# Definimos la funcion que extrae datos anidados de "cast"
def extraer_cast(list):
    try:
        if list != []:
            list_aux= ast.literal_eval(list)

            return ', '.join([dic['name'] for dic in list_aux])
        else: 
            return 'no register'
        
    except:
        return 'no register'

In [30]:
# Definimos la función extraer que filtra y devuelve el nombre del director
def extraer_director(crew_list):
    try:
        lista_aux = ast.literal_eval(crew_list) # Convierte la cadena de texto a una lista de diccionarios
        for dic in lista_aux:  # Busca y devuelve el nombre del director
            if dic.get('job') == 'Director':
                return dic.get('name', 'no registrado')
        return 'no registrado'  # Si no se encuentra un director, devuelve 'no registrado'
    except (ValueError, SyntaxError):
        return 'no registrado'

In [31]:
# Desanidamos "cast"

df_creditos['cast_clean'] =df_creditos['cast'].apply(extraer_cast)

In [32]:
#Desanidamos "crew" pero solo nos quedamos con el nombre de director

df_creditos['director'] =df_creditos['crew'].apply(extraer_director)

In [33]:
df_creditos.head()

,cast,crew,id,cast_clean,director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer


In [34]:
# Borramos las columnas innecesarias

df_creditos.drop(columns=['crew','cast'], inplace=True)

In [35]:
df_creditos.head()

,id,cast_clean,director
0,862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,15602,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,31357,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,11862,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer


In [36]:
df_creditos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          45476 non-null  int64 
 1   cast_clean  45476 non-null  object
 2   director    45476 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [37]:
df_mov.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44344 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   budget                      44344 non-null  float64       
 1   id                          44344 non-null  object        
 2   original_language           44344 non-null  object        
 3   overview                    44344 non-null  object        
 4   popularity                  44344 non-null  float64       
 5   release_date                44344 non-null  datetime64[ns]
 6   revenue                     44344 non-null  int32         
 7   runtime                     44344 non-null  float64       
 8   status                      44344 non-null  object        
 9   title                       44344 non-null  object        
 10  vote_average                44344 non-null  float64       
 11  vote_count                  44344 non-null  float64       


In [38]:
# Pasamos la columna "id" a integer

df_mov['id'] = pd.to_numeric(df_mov['id'],  errors='coerce', downcast='integer')

In [39]:
# Unimos los dos dataframes: "df_mov" y "df_creditos" creando un nuevo "dataset_unificado"

dataset_unificado = pd.merge(df_mov, df_creditos, on='id', how='inner')

In [40]:
dataset_unificado.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,title,...,vote_count,genres_name,companies_names,spoken_languages_clean,production_countries_clean,name_collection,return,release_year,cast_clean,director
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,...,5415.0,"Animation, Comedy, Family",Pixar Animation Studios,English,United States of America,Toy Story Collection,12.451801,1995,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104.0,Released,Jumanji,...,2413.0,"Adventure, Fantasy, Family","TriStar Pictures, Teitler Film, Interscope Com...","English, Français",United States of America,0,4.043035,1995,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0,101.0,Released,Grumpier Old Men,...,92.0,"Romance, Comedy","Warner Bros., Lancaster Gate",English,United States of America,Grumpy Old Men Collection,0.000000,1995,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156,127.0,Released,Waiting to Exhale,...,34.0,"Comedy, Drama, Romance",Twentieth Century Fox Film Corporation,English,United States of America,0,5.090760,1995,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911,106.0,Released,Father of the Bride Part II,...,173.0,Comedy,"Sandollar Productions, Touchstone Pictures",English,United States of America,Father of the Bride Collection,0.000000,1995,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer


In [41]:
dataset_unificado.shape

(44400, 21)

In [42]:
# Recortamos filas del dataset para reducirlo en mb y poder subirlo a Github, y lo guardamos en un archivo CSV.

dataset_modificado = dataset_unificado.head(20000).to_csv('C:/Users/ignac/OneDrive/Escritorio/Proyecto Individual 1/HenryLabs-PIMLO/dataset_modificado.csv' , index=False, header=True)